In [ ]:
#Connect Colab to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
sentiment_prompt = '[INST] <<SYS>>\nHãy xem mình là một Bot có thể phân loại cảm xúc của một câu tiếng Việt. Bot luôn đưa câu trả lời của mình ở dạng con số. Trong đó, giá trị 0 cho cảm xúc tiêu cực, 1 cho cảm xúc trung lập, 2 cho cảm xúc tích cực. Bot không được tự trả lời hay giả dạng thành Khách. Và đây là cuộc trò chuyện mới nhất giữa Bot và Khách. <</SYS>>\nHãy đọc kĩ và phân tích sentiment từ Khách. Sau đó, đưa ra câu trả lời của bạn dưới dạng list gồm 2 con số, trong đó số thứ nhất là Sentiment: `câu trả lời của bạn 0 (tiêu cực) hay 1 (trung lập) hay 2 (tích cực)` và số thứ hai là Confident_level: `độ tự tin cho câu trả lời của bạn trong khoảng từ 0 tới 1` \nKhách: "{context}"\nBot: [/INST]'
summary_prompt_full = "[INST] <<SYS>>\nBạn là một trợ lý hữu dụng, biết tôn trọng và thành thật. Bạn luôn luôn trả lời các câu hỏi một cách có ích nhiều nhất có thể, nhưng đồng thời phải an toàn. Câu trả lời của bạn không được bao gồm các ngôn từ độc hại, phân biệt chủng tộc, phân biệt giới tính, nguy hiểm, nội dung vi phạm pháp luật. Nhiệm vụ của bạn là tóm tắt đoạn văn bản nằm trong triplebacktick. Bài tóm tắt phải đầy đủ các thông tin quan trọng, ngắn gọn và thu hút người đọc. Ngôn ngữ bạn phải sử dụng để tóm tắt là tiếng Việt.\n<</SYS>>\n` ` `{document} ` ` ` [/INST]"
summary_medium_prompt = "[INST] <<SYS>>\nNhiệm vụ của bạn là tóm tắt đoạn văn bản sau, đưa ra câu trả lời là bản tóm tắt:\n<</SYS>>\n` ` `{document} ` ` ` [/INST]"
summary_weak_prompt = "[INST] Đoạn văn : {document}\nTóm tắt đoạn văn trên : [/INST]"

###TEST ON URA-LLAMA 7B

In [ ]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import time
pipeline_kwargs={
    "temperature": 1.0,
    "max_new_tokens": 500,
    "top_k": 1,
    "repetition_penalty": 1.1
}

In [ ]:
# Load model
from google.colab import userdata
secret_token = userdata.get('ura-llama-huggingface')
model = AutoModelForCausalLM.from_pretrained(
        "ura-hcmut/ura-llama-7b",
        device_map="auto",
        use_auth_token=secret_token,
        # trust_remote_code=True,
        # load_in_8bit=True,
        # low_cpu_mem_usage=True,
    )
model.config.pretraining_tp = 1
model.eval()

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
      

In [ ]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
   "ura-hcmut/ura-llama-7b",
    trust_remote_code=True,
    use_auth_token=secret_token,
)
tokenizer.pad_token = tokenizer.eos_token


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:810: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    task='text-generation',
    **pipeline_kwargs
)

Device set to use cuda:0


In [ ]:
query_template = "[INST] <<SYS>>\nBạn là một trợ lý thông minh.\n<</SYS>>\n\nHãy trả lời câu hỏi sau.\nCâu hỏi: {query}\nTrả lời: [/INST]"

In [ ]:
# while True:
#     # query = input("Query: ")
#     if query == "exit":
#         break
# query = "Tại sao nước biển lại mặn?"
# query = query_template.format(query=query)
# answer = pipeline(query)[0]["generated_text"]
# print(answer) #Too long
file_path = '/content/drive/MyDrive/personalize/sents.txt'
with open(file_path, 'r', encoding='utf-8') as file:
  lines = file.readlines()
for input_line in lines:
    # message = "Tại sao nước biển lại mặn?"
    query = sentiment_prompt.format(context=input_line)
    answer = pipeline(query)[0]["generated_text"]
    # print("> ", query, "\n")
    # For text responses, When the model's turn is complete it breaks out of the loop.
    with open('/content/drive/MyDrive/personalize/sentiment_out_ura.txt', 'a') as f:
      f.write(answer)
    print(answer)

KeyboardInterrupt: 

In [ ]:
folder_path = '/content/drive/MyDrive/personalize/details_input'
count = 1
for filename in os.listdir(folder_path):
  details_file_path = os.path.join(folder_path, filename)
  with open(details_file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
  # message = "Tại sao nước biển lại mặn?"
  query = summary_prompt_full.format(document=lines)
  # print("> ", query, "\n")
  answer = pipeline(query)[0]["generated_text"]
  with open(f'/content/drive/MyDrive/personalize/summary_out_ura/summary_out_normal_prompt{count}.txt', 'a') as f:
    f.write(answer)
  print(answer)
  count += 1

###TEST ON GEMINI 2.0

In [ ]:
!pip install -U -q google-genai

In [ ]:
#Generate Gemini API key to use
from google.colab import userdata
import os
import json
import time
os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')

In [ ]:
from google import genai
client = genai.Client(http_options= {'api_version': 'v1alpha'})

In [ ]:
MODEL = "gemini-2.0-flash-exp"

In [ ]:
import asyncio
import base64
import contextlib
import datetime
import os
import json
import wave
import itertools

from IPython.display import display, Audio

from google import genai
from google.genai import types

async def async_enumerate(it):
  n = 0
  async for item in it:
    yield n, item
    n +=1

In [ ]:
#{few_shot}\n
# sentiment_prompt = '[INST] <<SYS>>\nHãy xem mình là một Bot có thể phân loại cảm xúc của một câu tiếng Việt. Bot luôn đưa câu trả lời của mình ở dạng con số. Trong đó, giá trị 0 cho cảm xúc tiêu cực, 1 cho cảm xúc trung lập, 2 cho cảm xúc tích cực. Bot không được tự trả lời hay giả dạng thành Khách. Và đây là cuộc trò chuyện mới nhất giữa Bot và Khách. <</SYS>>\nHãy đọc kĩ và phân tích sentiment từ Khách. Sau đó, đưa ra câu trả lời của bạn dưới dạng json với định dạng là ``` json {"Sentiment": `câu trả lời của bạn 0 (tiêu cực) hay 1 (trung lập) hay 2 (tích cực)`, "confident_level": `độ tự tin cho câu trả lời của bạn trong khoảng từ 0 tới 1` } ` ` `\nKhách: "{context}"\nBot: [/INST]'
sentiment_prompt = '[INST] <<SYS>>\nHãy xem mình là một Bot có thể phân loại cảm xúc của một câu tiếng Việt. Bot luôn đưa câu trả lời của mình ở dạng con số. Trong đó, giá trị 0 cho cảm xúc tiêu cực, 1 cho cảm xúc trung lập, 2 cho cảm xúc tích cực. Bot không được tự trả lời hay giả dạng thành Khách. Và đây là cuộc trò chuyện mới nhất giữa Bot và Khách. <</SYS>>\nHãy đọc kĩ và phân tích sentiment từ Khách. Sau đó, đưa ra câu trả lời của bạn dưới dạng list gồm 2 con số, trong đó số thứ nhất là Sentiment: `câu trả lời của bạn 0 (tiêu cực) hay 1 (trung lập) hay 2 (tích cực)` và số thứ hai là Confident_level: `độ tự tin cho câu trả lời của bạn trong khoảng từ 0 tới 1` \nKhách: "{context}"\nBot: [/INST]'
summary_prompt_full = "[INST] <<SYS>>\nBạn là một trợ lý hữu dụng, biết tôn trọng và thành thật. Bạn luôn luôn trả lời các câu hỏi một cách có ích nhiều nhất có thể, nhưng đồng thời phải an toàn. Câu trả lời của bạn không được bao gồm các ngôn từ độc hại, phân biệt chủng tộc, phân biệt giới tính, nguy hiểm, nội dung vi phạm pháp luật. Nhiệm vụ của bạn là tóm tắt đoạn văn bản nằm trong triplebacktick. Bài tóm tắt phải đầy đủ các thông tin quan trọng, ngắn gọn và thu hút người đọc. Ngôn ngữ bạn phải sử dụng để tóm tắt là tiếng Việt.\n<</SYS>>\n` ` `{document} ` ` ` [/INST]"
summary_medium_prompt = "[INST] <<SYS>>\nNhiệm vụ của bạn là tóm tắt đoạn văn bản sau, đưa ra câu trả lời là bản tóm tắt:\n<</SYS>>\n` ` `{document} ` ` ` [/INST]"
summary_weak_prompt = "[INST] Đoạn văn : {document}\nTóm tắt đoạn văn trên : [/INST]"

In [ ]:
config={"generation_config": {"response_modalities": ["TEXT"]}}

In [ ]:
# input_lines = ["giờ thực hành quá ít.", "giáo viên vui tính , tận tâm ."]
file_path = '/content/drive/MyDrive/personalize/sents.txt'
with open(file_path, 'r', encoding='utf-8') as file:
  lines = file.readlines()
for input_line in lines:
  async with client.aio.live.connect(model=MODEL, config=config) as session:
    # message = "Tại sao nước biển lại mặn?"
    query = summary_prompt_full.format(document=input_line)
    # print("> ", query, "\n")
    await session.send(query, end_of_turn=True)
    # For text responses, When the model's turn is complete it breaks out of the loop.
    turn = session.receive()
    async for chunk in turn:
      with open('/content/drive/MyDrive/personalize/sentiment_out.txt', 'a') as f:
        if chunk.text is not None:
          # print(f'- {chunk.text}')
          f.write(chunk.text)
          print(chunk.text)
# async def main(input_line):
#     async with client.aio.live.connect(model=MODEL, config=config) as session:
#         # message = sentiment_prompt +
#         query = sentiment_prompt.format(context=input_line)
#         # print("> ", query, "\n")
#         await session.send(query, end_of_turn=True)

#     # For text responses, When the model's turn is complete it breaks out of the loop.
#     turn = session.receive()
#     async for chunk in turn:
#         if chunk.text is not None:
#             print(f'- {chunk.text}')


[
0, 0.8]

[
0, 0.9]

[
2, 0.9]

[
2, 0.9]

[
1, 0.8]

[
2, 0.9]

[
1, 0.7]

[
2, 0.9]

[
2, 0.9]

[
0, 0.8]

[
2, 0.9]

[
2, 0.9]

[
0, 0.9]

[
2, 0.9]

[
2, 0.9]

[
0, 0.8]

[
2, 0.9]

[
2, 0.9]

[
1, 0.9]

[
0, 0.9]

[
1, 0.8]

[
0, 0.9]

[
2, 0.9]

[
1, 0.7]

[
2, 0.9]

[
2, 0.95]

[
0, 0.9]

[
1, 0.8]

[
1, 0.7]

[
2, 0.9]

[
1, 0.8]

[
2, 0.9]

[
2, 0.95]

[
0, 0.8]

[
0, 0.8]

[
0, 0.9]

[
0, 0.8]

[
1, 0.7]

[
2, 0.9]

[
0, 0.9]

Bot
: [1, 0.8]

[
0, 0.9]

[
2, 0.9]

Bot
: [2, 0.9]

[
0, 0.8]

[
0, 0.8]

[
2, 0.9]

[
1, 0.7]

[
2, 0.9]

[
2, 0.9]

[
2, 0.9]

[
0, 0.8]

[
1, 0.7]

[
0, 0.9]

[
2, 0.9]

[
2, 0.9]

[
2, 0.8]

[
0, 0.8]

[
2, 0.95]

[
1, 0.7]

[
2, 0.9]

[
2, 0.9]

[
2, 0.9]

[
2, 0.7]

[
2, 0.9]

[
2, 0.9]

[
2, 0.8]

[
0, 0.9]

[
0, 0.9]

[
0, 0.9]

[
2, 0.9]

[
2, 0.7]

[
2, 0.8]

[
0, 0.8]

[
2, 0.95]

[
0, 0.9]

[
1, 0.8]

[
0, 0.9]

[
1, 0.7]

[
0, 0.9]

[
2, 0.9]

[
0, 0.8]

[
2, 0.95]

[
2, 0.9]

[
0, 0.9]

[
0, 0.8]

[
2, 0.9]

[
1, 0.8]

[
2, 0.9]

[
0, 0

In [ ]:
folder_path = '/content/drive/MyDrive/personalize/details_input'
count = 1
for filename in os.listdir(folder_path):
  details_file_path = os.path.join(folder_path, filename)
  with open(details_file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()
  async with client.aio.live.connect(model=MODEL, config=config) as session:
    # message = "Tại sao nước biển lại mặn?"
    query = summary_prompt_full.format(document=lines)
    # print("> ", query, "\n")
    await session.send(query, end_of_turn=True)
    # For text responses, When the model's turn is complete it breaks out of the loop.
    turn = session.receive()
    res = ""
    async for chunk in turn:
      if chunk.text is not None:
        # print(f'- {chunk.text}')
        res += chunk.text
    with open(f'/content/drive/MyDrive/personalize/summary_out/summary_out_normal_prompt{count}.txt', 'a') as f:
      f.write(res)
    print(res)
    count += 1
  time.sleep(5)

Vụ phá rừng nghiêm trọng tại Kon Tum đã được khởi tố. Cơ quan chức năng phát hiện 14 cây bị chặt hạ với hơn 12m3 gỗ, chủ yếu là thông thường. Nguyên nhân ban đầu được cho là do "lâm tặc" lợi dụng việc vận chuyển vật liệu xây dựng thủy điện để trà trộn khai thác gỗ. Hạt Kiểm lâm đã chuyển hồ sơ sang cơ quan điều tra. Để làm rõ trách nhiệm, 3 cán bộ liên quan đã bị tạm đình chỉ công tác. Tính từ đầu năm 2018, Kon Plông đã xảy ra 106 vụ vi phạm khai thác rừng, gây thiệt hại hơn 208m3 gỗ và 11,8ha rừng.

Ngày 14/5, tại thủy điện Sông Tranh 2, Quảng Nam, một người dân đã phát hiện thi thể không còn nguyên vẹn, trơ xương, không đầu, không chân và mặc quần jean ngắn màu xanh. Sự việc được báo cáo lên cơ quan chức năng để điều tra.

VKSND tỉnh Sóc Trăng đã tổ chức buổi xin lỗi công khai bà Huỳnh Ngọc Bích, Giám đốc HTX Thủ công mỹ nghệ Ngọc Bích, về việc khởi tố oan sai bà trong vụ án tham ô xảy ra tại Trung tâm Khuyến công tỉnh. Nguyên nhân được xác định là do lỗi của những người tiến hành tố

In [ ]:
import re

def extractive_fragments(A, S):
  """
  Hàm này trả về tập hợp các đoạn trích (extractive fragments) từ văn bản tóm tắt S
  được trích xuất từ văn bản đầy đủ A.

  Args:
    A: Văn bản đầy đủ (string).
    S: Văn bản tóm tắt (string).

  Returns:
    Một tập hợp các đoạn trích (set of strings).
  """

  A = A.lower()
  S = S.lower()

  fragments = set()
  i = 0
  j = 0

  while i < len(S):
    f = ""
    while j < len(A):
      if S[i] == A[j]:
        i_prime = i
        j_prime = j
        while i_prime < len(S) and j_prime < len(A) and S[i_prime] == A[j_prime]:
          i_prime = i_prime + 1
          j_prime = j_prime + 1

        if len(f) < (i_prime - i - 1):
          f = S[i:i_prime]
        j = j_prime
      else:
        j = j + 1

    i = i + max(len(f), 1)
    j = 0
    fragments.add(f)

  return fragments

def extractive_fragment_coverage(A, S):
  """
  Hàm này tính toán Extractive Fragment Coverage giữa A và S.

  Args:
    A: Văn bản đầy đủ (string).
    S: Văn bản tóm tắt (string).

  Returns:
    Extractive Fragment Coverage (float).
  """

  fragments = extractive_fragments(A, S)
  total_fragment_length = sum([len(re.findall(r'\w+', f)) for f in fragments])
  total_summary_length = len(re.findall(r'\w+', S))

  return total_fragment_length / total_summary_length

def extractive_fragment_density(A, S):
  """
  Hàm này tính toán Extractive Fragment Density giữa A và S.

  Args:
    A: Văn bản đầy đủ (string).
    S: Văn bản tóm tắt (string).

  Returns:
    Extractive Fragment Density (float).
  """

  fragments = extractive_fragments(A, S)
  total_squared_fragment_length = sum([len(re.findall(r'\w+', f))**2 for f in fragments])
  total_summary_length = len(re.findall(r'\w+', S))

  return total_squared_fragment_length / total_summary_length

def compression_ratio(A, S):
  """
  Hàm này tính toán Compression Ratio giữa A và S.

  Args:
    A: Văn bản đầy đủ (string).
    S: Văn bản tóm tắt (string).

  Returns:
    Compression Ratio (float).
  """

  total_article_length = len(re.findall(r'\w+', A))
  total_summary_length = len(re.findall(r'\w+', S))

  return total_article_length / total_summary_length

# Ví dụ sử dụng
A = "Đây là một văn bản đầy đủ. Nó chứa nhiều thông tin."
S = "Văn bản tóm tắt chứa thông tin."

coverage = extractive_fragment_coverage(A, S)
density = extractive_fragment_density(A, S)
compression = compression_ratio(A, S)

print(f"Extractive Fragment Coverage: {coverage}")
print(f"Extractive Fragment Density: {density}")
print(f"Compression Ratio: {compression}")

In [ ]:
lines = """ Bản_án cho đối_tượng giả_danh công_an để lừa_đảo
Theo lời khai của Huy tại phiên_toà , để có tiền sử_dụng cá_nhân , Huy “ nổ ” là sĩ_quan cục Phòng_chống ma_tuý của bộ Công_an đóng tại TP. Đà_Nẵng , có nguồn mua ô_tô thanh_lý giá rẻ , và khả_năng chạy việc vào ngành công_an .
Chỉ với lời “ nổ ” này , từ tháng 10/2016 đến 9/2017 , nhiều người đã bị lừa_đảo với tổng_số tiền 3,2 tỷ đồng .
Trong đó , người bị Huy lừa nhiều nhất là vợ_chồng ông Bảo_Th . , ngụ quận Hải_Châu .
Huy giới_thiệu với cặp vợ_chồng này mình có suất mua ô_tô thanh_lý giá rẻ và rủ họ mua cùng .
Tin lời , vợ_chồng ông Th .
đưa cho Huy hơn 1 tỷ đồng .
Cùng thủ_đoạn , Huy lừa thêm ông Nguyễn_Tấn_T. 970 triệu đồng , Lê_Quốc_Th .
400 triệu đồng , Trần_Nhật_S. 300 triệu đồng …
Sau chiêu_thức mua xe thanh_lý , Huy chuyển sang giả_vờ có khả_năng xin việc vào ngành công_an .
Với chiêu_thức này , Huy lừa vợ_chồng ông Đinh_Ngọc_H. 250 triệu đồng .
Ngoài_ra , Huy hứa_hẹn , tháng 3/2017 sẽ đưa kết_quả cho con ông H. đi làm_việc .
Tuy_nhiên , sau nhiều lần hẹn mà không có quyết_định tuyển_dụng , ông H. đã gửi đơn tố_cáo đến cơ_quan Công_an .
Từ đó , những hành_vi sai_trái của Huy lần_lượt được truy ra .
Huy tại phiên_toà ."""
async with client.aio.live.connect(model=MODEL, config=config) as session:
    # message = "Tại sao nước biển lại mặn?"
    query = summary_prompt_full.format(document=lines)
    # print("> ", query, "\n")
    await session.send(query, end_of_turn=True)
    # For text responses, When the model's turn is complete it breaks out of the loop.
    turn = session.receive()
    #Concat multiple chunk into 1 paragraph
    res = ""
    async for chunk in turn:
      if chunk.text is not None:
        # print(f'- {chunk.text}')
        res += chunk.text
    print(res)

Đối tượng Huy đã giả danh sĩ quan công an để lừa đảo chiếm đoạt 3,2 tỷ đồng của nhiều người từ tháng 10/2016 đến tháng 9/2017. Huy "nổ" có khả năng mua ô tô thanh lý giá rẻ và xin việc vào ngành công an, khiến nhiều người tin tưởng và giao tiền. Các nạn nhân gồm vợ chồng ông Bảo Th., ông Nguyễn Tấn T., Lê Quốc Th., Trần Nhật S., vợ chồng ông Đinh Ngọc H. Sau nhiều lần hứa hẹn không thành, các nạn nhân đã tố cáo Huy tới cơ quan công an.



###STUFF

In [ ]:
#Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Unzip file '/content/vietnews-master.zip'
import zipfile

with zipfile.ZipFile('/content/vietnews-master.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')


In [ ]:
import os

def extract_news_parts(file_path, abstract_file, details_file):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # Initialize placeholders
    title = ""
    abstract = ""
    content = []

    # Extract parts based on structure
    section = "title"
    for line in lines:
        line = line.strip()
        if not line:  # Skip empty lines
            if section == "title":
                section = "abstract"
            elif section == "abstract":
                section = "content"
            continue

        if section == "title":
            title += line
        elif section == "abstract":
            abstract += line
        elif section == "content":
            content.append(line)

    # Combine details into one part
    details = f"{title}\n" + "\n".join(content)

    # Write abstract to its file
    with open(abstract_file, 'w', encoding='utf-8') as file:
        file.write(abstract)

    # Write details to its file
    with open(details_file, 'w', encoding='utf-8') as file:
        file.write(details)

In [ ]:
folder_path = '/content'
folder_abstract_path = '/content/abstract'
folder_details_path = '/content/details'
#read files in folder_path
for filename in os.listdir(folder_path):
    if filename.endswith('.txt.seg'):
        file_path = os.path.join(folder_path, filename)
        abstract_file = os.path.join(folder_abstract_path, filename.replace('.txt.seg', '_abstract.txt'))
        details_file = os.path.join(folder_details_path, filename.replace('.txt.seg', '_details.txt'))
        extract_news_parts(file_path)
# print("Details:\n", details)
# print("\nSummary:\n", abstract)

Details:
 Bản_án cho đối_tượng giả_danh công_an để lừa_đảo
Theo lời khai của Huy tại phiên_toà , để có tiền sử_dụng cá_nhân , Huy “ nổ ” là sĩ_quan cục Phòng_chống ma_tuý của bộ Công_an đóng tại TP. Đà_Nẵng , có nguồn mua ô_tô thanh_lý giá rẻ , và khả_năng chạy việc vào ngành công_an .
Chỉ với lời “ nổ ” này , từ tháng 10/2016 đến 9/2017 , nhiều người đã bị lừa_đảo với tổng_số tiền 3,2 tỷ đồng .
Trong đó , người bị Huy lừa nhiều nhất là vợ_chồng ông Bảo_Th . , ngụ quận Hải_Châu .
Huy giới_thiệu với cặp vợ_chồng này mình có suất mua ô_tô thanh_lý giá rẻ và rủ họ mua cùng .
Tin lời , vợ_chồng ông Th .
đưa cho Huy hơn 1 tỷ đồng .
Cùng thủ_đoạn , Huy lừa thêm ông Nguyễn_Tấn_T. 970 triệu đồng , Lê_Quốc_Th .
400 triệu đồng , Trần_Nhật_S. 300 triệu đồng …
Sau chiêu_thức mua xe thanh_lý , Huy chuyển sang giả_vờ có khả_năng xin việc vào ngành công_an .
Với chiêu_thức này , Huy lừa vợ_chồng ông Đinh_Ngọc_H. 250 triệu đồng .
Ngoài_ra , Huy hứa_hẹn , tháng 3/2017 sẽ đưa kết_quả cho con ông H. đi l

In [ ]:
# Example usage
dataset_folder = "path_to_your_dataset"
output_folder = "processed_data"

os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(dataset_folder):
    if filename.endswith(".txt"):  # Assuming text files
        file_path = os.path.join(dataset_folder, filename)
        abstract, content = process_newspaper_file(file_path)

        # Save Abstract and Content separately
        with open(os.path.join(output_folder, f"{filename}_abstract.txt"), 'w', encoding='utf-8') as f_abstract:
            f_abstract.write(abstract)

        with open(os.path.join(output_folder, f"{filename}_input.txt"), 'w', encoding='utf-8') as f_content:
            f_content.write(content)